In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-midterm-p5/train.csv
/kaggle/input/2021-ml-midterm-p5/test.csv
/kaggle/input/2021-ml-midterm-p5/sample_submit.csv


In [2]:
# data load
train_data = pd.read_csv('/kaggle/input/2021-ml-midterm-p5/train.csv')
test_data = pd.read_csv('/kaggle/input/2021-ml-midterm-p5/test.csv')
submit = pd.read_csv('/kaggle/input/2021-ml-midterm-p5/sample_submit.csv')

print(train_data.head())
print(test_data.head())

   index  enrollee_id      city  city_development_index gender  \
0      0         8949  city_103                   0.920   Male   
1      1        29725   city_40                   0.776   Male   
2      3        33241  city_115                   0.789    NaN   
3      4          666  city_162                   0.767   Male   
4      5        21651  city_176                   0.764    NaN   

       relevent_experience enrolled_university education_level  \
0  Has relevent experience       no_enrollment        Graduate   
1   No relevent experience       no_enrollment        Graduate   
2   No relevent experience                 NaN        Graduate   
3  Has relevent experience       no_enrollment         Masters   
4  Has relevent experience    Part time course        Graduate   

  major_discipline experience company_size    company_type last_new_job  \
0             STEM        >20          NaN             NaN            1   
1             STEM         15        50-99         Pvt L

In [3]:
# x, y, test data를 구분해주었고, 필요없다고 판단한 x, test데이터의 열은 지움
x_data = train_data.drop(['index', 'target', 'city'], axis = 1)
y_data = train_data['target']
test_data = test_data.drop(['index', 'city'], axis = 1)

In [4]:
# x data information 보기
x_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             15326 non-null  int64  
 1   city_development_index  15326 non-null  float64
 2   gender                  11752 non-null  object 
 3   relevent_experience     15326 non-null  object 
 4   enrolled_university     15009 non-null  object 
 5   education_level         14964 non-null  object 
 6   major_discipline        13068 non-null  object 
 7   experience              15276 non-null  object 
 8   company_size            10612 non-null  object 
 9   company_type            10445 non-null  object 
 10  last_new_job            14983 non-null  object 
 11  training_hours          15326 non-null  int64  
dtypes: float64(1), int64(2), object(9)
memory usage: 1.4+ MB


In [5]:
# object형식인 것들을 labelencoder로 실수화 해줌
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() 

x_data['gender'] = le.fit_transform(x_data['gender'])
test_data['gender'] = le.transform(test_data['gender'])

x_data['relevent_experience'] = le.fit_transform(x_data['relevent_experience'])
test_data['relevent_experience'] = le.transform(test_data['relevent_experience'])

x_data['enrolled_university'] = le.fit_transform(x_data['enrolled_university'])
test_data['enrolled_university'] = le.transform(test_data['enrolled_university'])

x_data['education_level'] = le.fit_transform(x_data['education_level'])
test_data['education_level'] = le.transform(test_data['education_level'])

x_data['major_discipline'] = le.fit_transform(x_data['major_discipline'])
test_data['major_discipline'] = le.transform(test_data['major_discipline'])

x_data['experience'] = le.fit_transform(x_data['experience'])
test_data['experience'] = le.transform(test_data['experience'])

x_data['company_size'] = le.fit_transform(x_data['company_size'])
test_data['company_size'] = le.transform(test_data['company_size'])

x_data['last_new_job'] = le.fit_transform(x_data['last_new_job'])
test_data['last_new_job'] = le.transform(test_data['last_new_job'])

x_data['company_type'] = le.fit_transform(x_data['company_type'])
test_data['company_type'] = le.transform(test_data['company_type'])


In [6]:
# 데이터를 보고 표준화가 필요한 것 같다고 판단하여 scaler 사용 - data preprocessing
from sklearn.preprocessing import StandardScaler
scaler2 = StandardScaler()
x_data_std = scaler2.fit_transform(x_data)
test_data_std = scaler2.transform(test_data)

In [7]:
# logistic regression
from sklearn.linear_model import LogisticRegression
lgt = LogisticRegression(C = 0.1, max_iter = 1000)
lgt.fit(x_data_std, y_data)
print(lgt.score(x_data_std, y_data))

0.7669972595589195


In [8]:
# lda
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(x_data_std, y_data)
print(lda.score(x_data_std, y_data))

0.7677802427247814


In [9]:
# qda
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_data_std, y_data)
print(qda.score(x_data_std, y_data))

0.7694767062508157


In [10]:
# pred & submit
pred_qda = qda.predict(test_data_std)
submit['target'] = pred_qda
submit.to_csv('submit_qda.csv',index = False)

In [11]:
# 먼저 데이터를 로드 한 후 x, y, test 데이터로 나눠주고 index가 필요없다고 생각하여 index를 드랍하였다.
# 또한 object형이 너무많아서 object를 모두 실수화 해주었다. city index또한 중요하지 않다고 판단하여 드랍했다. 또한 데이터를 분포를 보니 전처리가 필요할 것 같아 표준화를 해주었다
# y데이터를 보니 범주형 데이터라 분류 문제라 판단하였고 분류 3개의 모델을 각각 학습해보고 예상 점수를  구해본 결과 qda가 가장 높게 나와 qda를선택했다.